In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words('indonesian') + list(punctuation)

# import data

In [3]:
df = pd.read_csv('data/dataText/data/kompas.csv')
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# extract TFIDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo)

In [7]:
tfidf_matrix = tfidf.fit_transform(df.teks)

# TFIDF Similarity -> document similarity

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [ ]:
sim.argsort()
# sorting berdasarkan index dari paling yang tidak mirip -> mirip

array([[ 932, 1131, 1593, ...,  215,  144,    0]], dtype=int64)

In [ ]:
import numpy as np

a = np.array([1, 5,4,3,6,2,3])
a.argmax()
# index brp untuk angka yang terbesar

4

In [12]:
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [13]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [14]:
df.teks[215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

In [15]:
df.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

# keyword extraction

In [16]:
vocab = tfidf.get_feature_names()
vocab[-10:]

c:\Users\akmal\anaconda3\envs\jcopml\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [17]:
tfidf_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [18]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274212, 365469, 365468, ..., 386379, 436652, 169219], dtype=int64)

In [19]:
vocab[169219]

'ginandjar'

In [20]:
vocab[436652]

'putusan'

In [21]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

In [22]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
    matrix = tfidf.transform([doc])
    vocab = tfidf.get_feature_names()

    sorted_tfidf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

In [23]:
text = '''Cagub nomor urut 1 Ridwan Kamil menyinggung penggusuran di Jakarta paling banyak dilakukan di era Basuki Tjahaja Purnama alias Ahok. Ridwan Kamil menjabarkan data penggusuran tahun 2016.
Awalnya Ridwan Kamil membantah pernyataan Cagub nomor urut 3 Pramono Anung yang ingin memanfaatkan lahan milik Pemprov Jakarta atau BUMD untuk dijadikan hunian warga. Menurut RK, program itu tidak memungkinkan.

"Yang pertama kalau saya jadi gubernur kami akan menggunakan lahan yang dimiliki pemerintah Jakarta dan BUMD untuk menjadi tempat hunian bagi warga Jakarta, kami akan mengembangkan kantor kecamatan sekolah untuk dibuat hunian ke atas," kata Pramono di debat ketiga Pilkada Jakarta, Minggu (17/11/2024).

Baca artikel detiknews, "RK: Gubernur Paling Banyak Menggusur dari Partainya Mas Pram, Pak Ahok" selengkapnya https://news.detik.com/pilkada/d-7643381/rk-gubernur-paling-banyak-menggusur-dari-partainya-mas-pram-pak-ahok.

Download Apps Detikcom Sekarang https://apps.detik.com/detik/'''

In [24]:
extract_keywords_tfidf(text, tfidf)

c:\Users\akmal\anaconda3\envs\jcopml\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['hunian',
 'kamil',
 'ridwan',
 'nomor urut',
 'hunian warga',
 'ahok',
 'rk',
 'urut',
 'bumd',
 'penggusuran']